In [11]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import time
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, filterFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2, figures)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output
from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
output_notebook()

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

Loading BokehJS ...

In [12]:
def sfun(x):
    return all([ ("Forsa Hartmoor" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("EWB" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("Pforzheim" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    #return all([ ("BMW Landshut 4.10" in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')
fleet.T;

In [13]:
motor = fleet.iloc[0]
modes = ['undefined','OFF','MAN','AUTO']
success = ['success','failed','undefined']
e=Engine.from_fleet(mp,motor)
#pp_from='2022-03-28 06:00' # 1 Start
#pp_to='2022-03-28 08:14'
pp_from='2022-03-01' # 15 Starts
pp_to='2022-03-07'
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

('Forsa Hartmoor M01',
 ['undefined', 'OFF', 'MAN', 'AUTO'],
 ['success', 'failed', 'undefined'])

In [14]:
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=True, silent=False, debug=False)
fsm.run1(silent=False, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent = False)

FSM2: 100%|████████████████████████████████| 15/15 [00:00<00:00, 15.62 starts/s]


In [15]:
rdf = fsm.starts
rda = rdf[:].reset_index(drop='index')
rda = rda[(rda['mode'].isin(modes) & rda['success'].isin(success))].reset_index(drop='index')
rdb = rda
von_dt=pd.to_datetime(fsm.first_message); von=int(von_dt.timestamp())
bis_dt=pd.to_datetime(fsm.last_message); bis=int(bis_dt.timestamp())
print(f"von: {von_dt.strftime('%d.%m.%Y %H:%M:%S')} --- bis: {bis_dt.strftime('%d.%m.%Y %H:%M:%S')}")
print(f"Starts: {rdf.shape[0]}, Successful: {rdf[rdf['success'] == 'success'].shape[0]}, Failed: {rdf[rdf['success'] == 'failed'].shape[0]}, Undefined: {rdf[rdf['success'] == 'undefined'].shape[0]} => ")
print(f"Starting reliability raw: {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0])*100.0:3.1f}% ")
print(f"Starting reliability: {rdf[rdf['success'] == 'success'].shape[0]/(rdf.shape[0]-rdf[rdf['success'] == 'undefined'].shape[0])*100.0:3.1f}% ")
pd.DataFrame.from_dict(e.dash, orient='index').T

von: 01.03.2022 00:53:55 --- bis: 06.03.2022 23:54:26
Starts: 15, Successful: 11, Failed: 2, Undefined: 2 => 
Starting reliability raw: 73.3% 
Starting reliability: 84.6% 


,Name,Engine ID,Design Number,Engine Type,Engine Version,P,P_nom,BMEP,serialNumber,id,Count_OpHour,val start,oph@start,oph parts,LOC
0,Forsa Hartmoor M01,M01,AL87,624,H12,24,4495.0,24.5,1486144,159396,297.0,2022-01-10,63,234.0,0.162


In [16]:
#rda = rda[rda['count_alarms'] > 0]
rda[filterFSM.run2filter_content].round(2).fillna('')

,no,success,mode,startpreparation,starter,speedup,idle,synchronize,loadramp,cumstarttime,targetload,ramprate,targetoperation,rampdown,coolrun,runout,count_alarms,count_warnings
0,0,success,undefined,102.49,4.34,21.49,4.34,58.83,87.02,278.52,4515.61,1.15,7221.01,126.79,9.99,50.68,0,0
1,1,success,undefined,98.36,3.63,19.08,4.24,57.12,96.29,278.72,4498.97,1.04,4326.78,127.28,9.97,50.68,0,0
2,2,success,undefined,101.79,4.24,19.48,4.54,64.56,83.60,278.20,4500.06,1.2,12621.82,126.53,10.07,50.55,0,0
3,3,success,undefined,103.53,4.04,23.14,4.55,48.63,95.88,279.77,4512.82,1.05,8030.82,126.76,10.09,50.85,0,0
4,4,success,undefined,105.21,4.34,22.93,4.76,60.43,80.82,278.48,4495.46,1.24,9020.71,127.00,9.98,50.85,0,0
5,5,success,undefined,60.54,3.84,18.27,4.23,40.45,150.04,277.37,4481.0,0.66,623.4,126.96,9.99,50.86,0,0
6,6,success,undefined,60.54,3.84,18.27,4.23,40.45,148.49,275.82,,,624.95,126.96,9.99,50.86,0,0
7,7,success,undefined,102.09,3.84,22.61,4.64,2.01,144.09,279.28,4511.39,0.7,7220.11,126.92,10.00,50.83,0,0
8,8,success,undefined,97.74,3.83,19.90,4.44,62.37,89.52,277.81,4492.7,1.12,12620.02,126.80,9.99,50.73,0,0
9,9,failed,undefined,101.98,3.63,23.00,4.54,61.97,32.38,227.51,2006.46,1.38,,0.30,45.02,,1,1


In [17]:
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success']) & (rda.targetload > 2800.0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda.count_alarms > 0)].copy()
#rde = rda[(rda.starttime > fsm._e['Commissioning Date']) & (rda['success'])].copy()
rde = rda
rde['datetime'] = pd.to_datetime(rde['starttime'])
sdict ={'success':1, 'failed':0, 'undefined':-1}
rde['isuccess'] = rde.apply(lambda x: sdict[x['success']], axis=1)
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout','isuccess']
display(rde[vec].describe().round(2))
dfigsize = (20,12)
dset = [
    {'col':['cumstarttime'],'_ylim':(-600,800), 'color':'darkblue'},
    {'col':['runout'],'_ylim':(0,100) },
    {'col':['targetload'],'_ylim':(-4000,26000) },
    {'col':['ramprate'],'_ylim':(-5,7)},
    {'col':['loadramp'],'_ylim':(-150,900), 'color':'red'},
    {'col':['speedup'],'_ylim':(-100,200), 'color':'orange'},
    {'col':['synchronize'],'_ylim':(-20,400)},
    {'col':['startpreparation'],'_ylim':(-1000,800)},
    {'col':['count_warnings','count_alarms','isuccess'],'_ylim':(-1,200), 'color':['rgba(255,165,0,0.3)','rgba(255,0,0,0.3)','rgba(0,128,0,0.2)'] },
    {'col':['no'],'_ylim':(0,1000), 'color':['rgba(0,0,0,0.1)'] },
    ]
dset = equal_adjust(dset, rde, do_not_adjust=[-1], debug=False)
ftitle = f"{fsm._e}"
fig = dbokeh_chart(rde, dset, style='both', figsize=dfigsize ,title=ftitle);
bokeh_show(fig)

,startpreparation,speedup,idle,synchronize,loadramp,targetload,ramprate,cumstarttime,targetoperation,rampdown,coolrun,runout,isuccess
count,15.00,15.00,15.00,15.00,15.00,11.00,11.00,15.00,13.00,15.00,15.00,13.00,15.00
mean,94.57,20.87,4.43,49.88,97.75,4274.58,1.04,271.39,5090.44,109.99,14.67,50.72,0.60
std,14.26,1.75,0.17,15.71,34.73,752.32,0.22,17.56,4636.90,44.53,12.32,0.12,0.74
min,60.54,18.27,4.23,2.01,32.38,2006.46,0.66,227.51,110.97,0.30,9.97,50.55,-1.00
25%,95.37,19.69,4.24,43.75,85.31,4494.08,0.94,276.59,624.95,126.75,9.98,50.68,0.50
50%,98.36,20.83,4.44,50.98,96.29,4498.97,1.05,278.15,4326.78,126.80,9.99,50.68,1.00
75%,102.03,22.77,4.54,61.20,111.37,4509.73,1.18,278.62,8030.82,126.94,10.04,50.85,1.00
max,105.21,23.14,4.76,64.56,150.04,4515.61,1.38,279.77,12621.82,127.28,45.02,50.86,1.00


In [18]:
out = widgets.Output()
pfigsize=(20,12)

def myfigures(e):
    return {
    'actors' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
        ],
        'tecjet' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':['TecJet_GasPress1'],'_ylim': [0, 3], 'color':'rgba(255,0,0,0.4)', 'unit':'mbar'},
        {'col':['TecJet_GasTemp1'],'_ylim': [0, 3], 'color':'rgba(255,0,255,0.4)', 'unit':'°C'},
        {'col':['TecJet_GasDiffPress'],'_ylim': [0, 3], 'color':'rgba(0,255,0,0.4)', 'unit':'mbar'},
        ],
        'lubrication' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
        {'col':['Hyd_PressOilDif'],'ylim': [0, 3], 'color':'black', 'unit': 'bar'},
        {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
        {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
        ],
        'exhaust' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
        {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 12000], 'unit':'mV'},
        ],
        'ignition' : [
        {'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['lightblue','red'], 'unit':'kW'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
        {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
        {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
        {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
        {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
        ],    
    }

def update_fig(x=0):
    global vv; 
    rdbs = rdb[rdb.no == x]
    if not rdbs.empty:
        startversuch = rdbs.iloc[0]
        #startversuch = rdb.iloc[x]
        vv = startversuch.no; 
        #ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
        #display(HTML(ftitle));
        #disp_result(startversuch)

        # PLotter
        lfigures = myfigures(fsm._e)
        plotdef, vset = cplotdef(mp, lfigures)
        #dmaxlength = 1800
        dmaxlength = None
        dminlength = None
        #startversuch = rdb.iloc[vv]
        ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
        data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        fig_handles = []
        for doplot in plotdef:
            dset = lfigures[doplot]
            ltitle = f"{ftitle} | {doplot}"
            if count_columns(dset) > 12: # no legend, if too many lines.
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=pfigsize)
            else:
                fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=pfigsize)

            fig = FSM_add_Notations(fig, fsm, startversuch)
            disp_alarms(startversuch)
            disp_warnings(startversuch)
            fig = FSM_add_Alarms(fig, fsm, startversuch)
            fig = FSM_add_Warnings(fig, fsm, startversuch)
            fig_handles.append(bokeh_show(fig, notebook_handle=True))
        for h in fig_handles:
            push_notebook(handle=h)
    else:
        print(f"Start No {x} is not in the filtered results. please consider changing filters, if you want to access it.")

In [19]:
vv = 0
@out.capture(clear_output=True)
def cb(but):
    update_fig(_a.value)

def html_summary(*args):
    sv = rdf.iloc[_a.value]
    global vv; vv = _a.value;
    s = '''
    <style>
        table, td, th {
            border: 1px solid grey;
            border-collapse: collapse;
            padding: 0px 5px;
            text-align: right;
        }
    </style>
    '''
    summary = pd.DataFrame(sv[filterFSM.run2filter_content]).T
    r = f"{s}{summary.to_html(escape=False, index=False)}"
    _e.value = r
    #return r 
    
_a = widgets.IntText(description='No: ',layout=widgets.Layout(width='10%'))
_a.observe(html_summary, 'value')
_b = widgets.IntSlider(0, 0, rdf.shape[0]-1 , 1, layout=widgets.Layout(width='70%'))
mylink = widgets.jslink((_a, 'value'), (_b, 'value'))
_d = widgets.Button(description='Show Plots',disabled=False, button_style='primary')
_d.on_click(cb)
_e = widgets.HTML()
html_summary()

display( widgets.VBox([widgets.HBox([_b, _a, _d]),_e, out]));

In [ ]:
update_fig(vv)

In [ ]:
def f(x=0):
    global vv; vv = x; startversuch = rdb.iloc[x]
    ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')} CumStart: {startversuch['cumstarttime']:0.1f}"
    display(HTML(ftitle));
    disp_result(startversuch)
interact(f, x = IntSlider(0, 0, rdb.shape[0]-1 , 1, layout=widgets.Layout(width='90%')));

In [ ]:
# PLotter
for f in figures:
    print(f, end=', ')
data = pd.DataFrame([])
for doplot in ['actors','lubrication']:
#for doplot in ['actors']:
    dmaxlength = None
    dminlength = None
    #dmaxlength = 1200
    #dminlength = 200000
    dset = figures[doplot]
    vset = cvset(mp,dset)
    startversuch = rdb.iloc[vv]; 
    try:
        data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=cvset(mp,dset))
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        fig = FSM_splot(fsm, startversuch, data, dset, figsize=(18,10))
        fig = FSM_add_Notations(fig, fsm, startversuch)
        disp_alarms(startversuch)
        disp_warnings(startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    except Exception as err:
        print(err)

In [ ]:

def xwhere(data,key,level):
    return data.iloc[data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0].index]

def xwhere2(data,key,level):
    pts = data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0]
    newlist =[x for p in list(pts.index) for x in [p-1,p]]
    return data.iloc[newlist]

def xwhere_dir(data,key,level,direction):
    data['diff'] = pd.Series(np.diff(data[key].values))
    pts = data['datetime'][1:][np.array(data[key][1:]-level) * np.array(data[key][:-1]-level) < 0]
    if direction == 'up':
        erg = data.iloc[pts.index][data['diff'] > 0]
    elif direction == 'down': 
        erg = data.iloc[pts.index][data['diff'] < 0]
    else:
        erg = data.iloc[pts.index]
    return erg

xwhere_dir(data,'Various_Values_SpeedAct',50,'up')

In [ ]:
op0 = xwhere(data,'Hyd_PressOil',1.1)
op1 = xwhere(data,'Hyd_PressOil',0.1)
op2 = xwhere(data,'Hyd_PressOil',0.5)
prelube_degasing_time = (op0.iloc[0]['time'] - op1.iloc[0]['time']) / 1000.0
post_lube_time = (op1.iloc[1]['time'] - op2.iloc[1]['time']) / (1000.0 * 60) 
prelube_degasing_time, 'sec' , post_lube_time, 'min'

In [ ]:
xwhere(data,'Hyd_PressOil',0.5)

In [ ]:
xwhere(data,'Hyd_PressOil',0.1)

In [ ]:
xwhere(data,'Various_Values_SpeedAct',1503)

In [ ]:
def cdata(startversuch, data, phases):
    ts = int(startversuch['startstoptiming'][phases[0]][0]['start'].timestamp() * 1000)
    te = int(startversuch['startstoptiming'][phases[-1]][0]['end'].timestamp() * 1000)
    return data[(data.time > ts) & (data.time < te)].reset_index(drop=True)

def extract_data(startversuch, data, dataItemName, phases):
    ts = int(startversuch['startstoptiming'][phases[0]][0]['start'].timestamp() * 1000)
    te = int(startversuch['startstoptiming'][phases[-1]][0]['end'].timestamp() * 1000)
    return data[['datetime',dataItemName]][(data.time > ts) & (data.time < te)].reset_index(drop=True)

def edges_in_phase2(startversuch, data, dataItemName, phases, direction, ax=None):
    c = extract_data(startversuch, data, dataItemName, phases)
    c['_diff'] = pd.Series(np.diff(c[dataItemName].values))
    pts = xwhere_dir(c, '_diff', c['_diff'].max() * 0.1, direction)
    if ax is not None:
        ax.plot(c['datetime'], c[dataItemName])
        for i,p in pts.iterrows():
            ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
        ax.grid()        
    return pts

def FSM_pre_phases(startversuch, data, dataItemName, phases, direction, fig=None, y_pos=2, color='rgba(0,0,0,0.8)', line='solid', alpha=1.0, perc=0.11):
    c = extract_data(startversuch, data, dataItemName, phases)
    c['_diff'] = pd.Series(np.diff(c[dataItemName].values))
    if direction == 'up':
        tresh_hold = c['_diff'].max()
    elif direction == 'down':
        tresh_hold = c['_diff'].min()
    pts = xwhere_dir(c, '_diff', tresh_hold * perc, direction)
    if fig is not None:
        for i,p in pts.iterrows():
            FSM_VLine(fig, txt=f"{p['datetime'].strftime('%Y-%m-%d %H:%M:%S')} {dataItemName}", x_pos=p['datetime'], y_pos=y_pos, color=color, line=line, alpha=alpha)
            #ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
    return fig

def FSM_start(startversuch, data, dataItemName, phases, fig=None, y_pos=2, color='rgba(0,0,0,0.8)', line='solid', alpha=1.0, perc=0.11):
    c = extract_data(startversuch, data, dataItemName, phases)
    pts = xwhere_dir(c, dataItemName, c[dataItemName].max() * perc, 'up')
    if fig is not None:
        for i,p in pts.iterrows():
            FSM_VLine(fig, txt=f"{p['datetime'].strftime('%Y-%m-%d %H:%M:%S')} {dataItemName}", x_pos=p['datetime'], y_pos=y_pos, color=color, line=line, alpha=alpha)
            #ax.axvline(p['datetime'],linestyle='--', c='k', alpha=0.8, linewidth=1)
    return fig

d0 = cdata(startversuch, data, ['startpreparation','starter','speedup','idle','synchronize','targetoperation']).iloc[:400,:]
d = cdata(startversuch, data, ['startpreparation','starter','speedup'])

#fig = plt.figure(figsize=(12,8))
#ax = fig.add_subplot()
#ax.plot('datetime','Various_Values_SpeedAct', data=d)
#play(startversuch, data, 'Hyd_PressOil',['startpreparation','starter','speedup'],'up', ax)

In [ ]:
def smooth(y,box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

def smooth_fft(y, cut, dt):
    w = scipy.fftpack.rfft(y)
    f = scipy.fftpack.rfftfreq(len(y),dt)
    spectrum = w**2
    cutoff_idx = spectrum < (spectrum.max()/cut)
    w2 = w.copy()
    w2[cutoff_idx] = 0
    return scipy.fftpack.irfft(w2)

In [ ]:

#d0['gHyd_PressOilDif'] = pd.Series(savitzky_golay(np.array(d0['Hyd_PressOilDif']), 51, 3)) # window size 51, polynomial order 3
#d0['gHyd_PressOilDif'] = pd.Series(savitzky_golay(np.array(d0['Hyd_PressOilDif']), 29, 3)) # window size 51, polynomial order 3
d0['gHyd_PressOilDif'] = pd.Series(savgol_filter(np.array(d0['Hyd_PressOilDif']), 29, 3)) # window size 51, polynomial order 3
d0['hHyd_PressOilDif'] = pd.Series(smooth(np.array(d0['Hyd_PressOilDif']), 13))
d0['iHyd_PressOilDif'] = pd.Series(smooth_fft(np.array(d0['Hyd_PressOilDif']), 5000, 1))
fig = plt.figure(figsize=(16,12))
ax = fig.add_subplot()
ax.plot(d0['datetime'],d0['Hyd_PressOilDif'], 'b.', alpha=0.6)
ax.plot(d0['datetime'],d0['hHyd_PressOilDif'], 'g-', lw=4)
ax.plot(d0['datetime'],d0['gHyd_PressOilDif'], 'r-', lw=2)
ax.plot(d0['datetime'],d0['iHyd_PressOilDif'], 'k-', lw=2)
ax.grid()


In [ ]:
#np.array(d0['Hyd_PressOilDif'])
data

In [ ]:
dset = figures['lubrication']
vset = cvset(mp,dset)
fig = FSM_splot(fsm, startversuch, d0, dset, figsize=(18,10))
fig = FSM_add_Notations(fig, fsm, startversuch)
fig = FSM_add_Alarms(fig, fsm, startversuch)
fig = FSM_add_Warnings(fig, fsm, startversuch)
#fig = FSM_pre_phases(startversuch, data, 'Hyd_PressOil',['startpreparation','starter'],'up', fig, y_pos=50, color='brown')
fig = FSM_start(startversuch, data, 'Hyd_PressOil',['startpreparation'], fig, y_pos=50, color='brown',perc=0.05)
fig = FSM_start(startversuch, data, 'Various_Values_SpeedAct',['startpreparation','starter'], fig, y_pos=50, color='blue',perc=0.05)
#fig = FSM_pre_phases(startversuch, data, 'Various_Values_SpeedAct',['startpreparation','starter','speedup'],'up', fig, y_pos=250, color='blue')
bokeh_show(fig)

In [ ]:
# Algorithm visualization
dset2 = [{'col':['Power_PowerAct','helpline'], 'ylim':(-1000,12000), 'color':['red','rgba(0,128,0,0.2)'], 'unit':'kW'},
         {'col':['power_diff','power_diff_help'], '_ylim':(0,5000), 'color':['rgba(255,0,0,0.2)','rgba(0,128,0,0.2)'], 'unit':'kW/s'}]
ratedload = fsm._e['Power_PowerNominal']
interestingcycles = [vv] # Bautzen M01 T902
periodfactor = 3 # times calculated period length
helplinefactor = 0.8 # overlay slope to -fac * end value 
for i in interestingcycles: 
    #startversuch=fsm.results['starts'][i]
    startversuch=rdb.iloc[i]
    disp_result(startversuch)
    sno = startversuch['no']
    data, xmax, ymax, duration, ramprate = dmyplant2.loadramp_edge_detect(fsm,startversuch, periodfactor=periodfactor, helplinefactor=helplinefactor)
    if not data.empty:
        print(f"Start: {startversuch['no']:3d} xmax: {xmax}, ymax: {ymax:6.0f}, duration: {duration:5.1f}, ramprate: {ramprate / ratedload * 100.0:4.2f} %/s")
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        #data['power_diff_help'] = pd.Series(np.gradient(data['helpline']))
        fig = FSM_splot(fsm, startversuch, data, dset2, figsize=(16,8))
        fig.add_layout(Span(location=0.0,dimension='width',x_range_name='default', y_range_name='0',line_color='black', line_dash='solid', line_alpha=0.4)) 
        fig.add_layout(Span(location=ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig.add_layout(Span(location=xmax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig = FSM_add_Notations(fig, fsm, startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    else:
        print(f"Start: {startversuch['no']:3d} no data, no improvement possible.")

In [ ]:
fsm.results['serviceselectortiming'];

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
    print(f"{i:3} {v}")

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=False)):
    print(f"{i:3} {v}")

In [ ]:
rdb = rda
vec = ['startpreparation','speedup','idle','synchronize','loadramp','targetload','ramprate','cumstarttime','targetoperation','rampdown','coolrun','runout']
display(_=rdb[vec].hist(bins=30,figsize=(20,20)))
#ax_list[0][2].set_xlim((0,10))
#display(rdb[vec].hist(bins=30,figsize=(20,20)))
display(rdb[vec].describe().round(2))

In [ ]:
dset2 = [{'col':['Power_PowerAct','helpline'], 'ylim':(-1000,12000), 'color':['red','rgba(0,128,0,0.2)'], 'unit':'kW'},
         {'col':['power_diff','power_diff_help'], '_ylim':(0,5000), 'color':['rgba(255,0,0,0.2)','rgba(0,128,0,0.2)'], 'unit':'kW/s'}]
ratedload = fsm._e['Power_PowerNominal']
periodfactor = 3 # times calculated period length
helplinefactor = 0.8 # overlay slope to -fac * end value 
t0 = time.time()
for i, startversuch in rdb[-1:].iterrows() : 
#for i, startversuch in tqdm(rdb.iterrows(), total=rdb.shape[0], ncols=80, mininterval=1, unit=' starts', desc="FSM Run2"):
    data, xmax, ymax, duration, ramprate = dmyplant2.loadramp_edge_detect(fsm,startversuch, periodfactor, helplinefactor)
    if not data.empty:
        print(f"Start: {startversuch['no']:3d} xmax: {xmax}, ymax: {ymax:6.0f}, duration: {duration:5.1f}, ramprate: {ramprate / ratedload * 100.0:4.2f} %/s")
        data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
        #data['power_diff_help'] = pd.Series(np.gradient(data['helpline']))
        fig = FSM_splot(fsm, startversuch, data, dset2, figsize=(14,6))
        fig.add_layout(Span(location=0.0,dimension='width',x_range_name='default', y_range_name='0',line_color='black', line_dash='solid', line_alpha=0.4)) 
        fig.add_layout(Span(location=ymax,dimension='width',x_range_name='default', y_range_name='0',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig.add_layout(Span(location=xmax,dimension='height',line_color='blueviolet', line_dash='dashdot', line_alpha=0.4, line_width=2)) 
        fig = FSM_add_Notations(fig, fsm, startversuch)
        #fig = FSM_add_Alarms(fig, fsm, startversuch)
        #fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    else:
        print(f"Start: {startversuch['no']:3d} no data, no improvement possible.")
t1 = time.time()
print(f"{(t1-t0):4.1f} sec")    

In [ ]:
mfn = e._fname + '_messages.txt'
fsm.save_messages(mfn)
print(mfn)